In [76]:
import numpy as np 
import pandas as pd

In [77]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [78]:
df_test.drop(['Ticket'],axis=1,inplace= True)
df_test.drop(['Name'],axis=1,inplace= True)
df_train.drop(['Ticket'],axis=1,inplace= True)
df_train.drop(['Name'],axis=1,inplace= True)
df_train.drop(['Cabin'],axis=1,inplace= True)
df_test.drop(['Cabin'],axis=1,inplace= True)

In [79]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer

In [80]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [81]:
pipe1= Pipeline(steps=[('si',SimpleImputer(strategy='most_frequent' )),
('ohe',OneHotEncoder(sparse_output=False, drop='first'))])

In [82]:
pipe2 = Pipeline(steps=[('si2',SimpleImputer(strategy='mean'))])

In [83]:
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,NaN,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [84]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [85]:
ct = ColumnTransformer(transformers=[
    ('pipe1',pipe1,['Sex','Embarked']),
    ('pipe2',pipe2,['Age'])])

In [86]:
from xgboost import XGBRegressor
pipe = Pipeline(steps=[
    ('preprocess', ct),
    ('model', XGBRegressor(
    max_depth=4,
    n_estimators=200,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
))
])

In [87]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df_train.drop('Survived',axis=1),df_train['Survived'], test_size=0.8)

In [88]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('pipe1',
                                                  Pipeline(steps=[('si',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False))]),
                                                  ['Sex', 'Embarked']),
                                                 ('pipe2',
                                                  Pipeline(steps=[('si2',
                                                                   SimpleImputer())]),
                                                  ['Age'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=4, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=200, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [89]:
y_predict= pipe.predict(X_test)

In [90]:
y_predict

array([ 0.65274197,  0.03946348, -0.00421753,  0.32613954,  0.03875982,
        0.08882716, -0.00421753,  0.6587773 ,  1.0365326 ,  0.55891764,
        0.08356356,  0.7540208 , -0.00421753, -0.17798303,  0.08882716,
        0.20567857,  0.20097205,  0.6648733 ,  0.00914478,  0.08173934,
        0.48914778,  0.6798069 ,  0.80698425,  0.6648733 ,  1.0671748 ,
        0.48914778,  0.82067937,  0.58806145,  0.19817477,  0.9983403 ,
        0.20567857,  0.6082581 ,  0.02746792,  0.9983403 ,  0.00763258,
        0.6587773 ,  0.8315128 ,  0.19722974,  0.47014773,  0.08356356,
        0.00763258,  0.08356356,  0.5388736 ,  0.00914478,  0.6049216 ,
        0.79655486,  0.06052571,  0.03875982,  0.6049216 ,  0.22317058,
        0.1380175 ,  0.23394924,  0.00763258,  0.9690122 ,  0.08882716,
        0.07841754,  0.18524434,  0.42059365,  0.03875982,  0.40589425,
        0.5784127 ,  0.48914778,  0.6648733 ,  0.82067937,  0.79655486,
        0.19817477,  0.25500703,  0.22497322,  0.47014773,  0.65

In [91]:
y_predict = (y_predict >= 0.4).astype(int)

In [92]:
y_predict

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,

In [93]:

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print("Accuracy:", accuracy_score(y_test, y_predict))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_predict))
print("\nClassification Report:\n", classification_report(y_test, y_predict))

Accuracy: 0.7166900420757363
Confusion Matrix:
 [[310 132]
 [ 70 201]]

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.70      0.75       442
           1       0.60      0.74      0.67       271

    accuracy                           0.72       713
   macro avg       0.71      0.72      0.71       713
weighted avg       0.74      0.72      0.72       713



In [94]:
df_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500,S
414,1306,1,female,39.0,0,0,108.9000,C
415,1307,3,male,38.5,0,0,7.2500,S
416,1308,3,male,NaN,0,0,8.0500,S


In [96]:
y_test_predict = pipe.predict(df_test)

In [97]:
y_test_predict

array([-1.84117779e-01,  8.85851502e-01, -6.89051524e-02,  5.88061452e-01,
        6.64873302e-01,  9.68594253e-02,  8.64158928e-01,  2.23170578e-01,
        5.58917642e-01,  7.37421140e-02,  5.90287209e-01,  7.96554863e-01,
        5.38977742e-01,  7.80076012e-02,  8.85851502e-01,  1.03610396e+00,
        5.39577715e-02,  2.33334124e-01,  9.29836929e-01,  9.03632164e-01,
        2.91328412e-02,  4.09444928e-01,  1.00750089e+00,  2.33334124e-01,
        7.63718426e-01,  7.63257733e-03,  1.00030720e+00,  1.88083336e-01,
        2.80370533e-01,  5.78412712e-01,  7.63257733e-03,  3.87598239e-02,
        8.70065391e-01,  1.00750089e+00,  5.78412712e-01,  7.81933144e-02,
        1.00750089e+00,  7.54020810e-01,  2.09120840e-01,  5.90287209e-01,
        3.56959850e-01,  5.90287209e-01,  2.80370533e-01,  1.00750089e+00,
        1.11077034e+00,  2.09120840e-01,  8.04447591e-01,  3.78605843e-01,
        6.10973656e-01,  9.51463640e-01,  3.87598239e-02,  9.71916199e-01,
        8.91718447e-01,  

In [98]:
y_test_predict = (y_test_predict>=0.4).astype(int)

In [99]:
y_test_predict

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,

In [101]:
submission = pd.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": y_test_predict 
})
submission.to_csv("submission.csv", index=False)